In [1]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

# VGG11 simple (pas de BatchNorm, pas de dropout)
def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_L4R9AI_V6")
    model.add(layers.Input(shape=input_shape))

    # Bloc 1
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.SpatialDropout2D(0.15))

    # Bloc 2
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.SpatialDropout2D(0.20))

    # Nouveau bloc 2.5
    model.add(layers.Conv2D(96, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.SpatialDropout2D(0.25))

    # Bloc 3
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.SpatialDropout2D(0.25))

    # Fully connected
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='softmax'))

    return model


# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)

    # Optimiseur simple
    opt = tf.keras.optimizers.Adam(learning_rate=0.0008)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    t = timer()
    t.tic()

    reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.6,
    patience=3,
    verbose=1,
    min_lr=1e-5
    )
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        verbose=2,
        callbacks=[reduce_lr]
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)

    # Sauvegarde
    model.save("CIFAR10_VGG11_simple.h5")
    np.save("CIFAR10_xtest.npy", data.x_test)
    np.save("CIFAR10_ytest.npy", data.y_test)



✅ GPU détecté : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Train: (50000, 32, 32, 3) (50000, 10)
Test:  (10000, 32, 32, 3) (10000, 10)
Epochs: 50  Batch size: 64
➡️ Construction du modèle...


Model: "VGG11_L4R9AI_V6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d               │ (None, 16, 16, 32)     │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_1             │ (None, 8, 8, 64)       │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 96)       │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 96)       │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_2             │ (None, 4, 4, 96)       │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 4, 4, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 390,346 (1.49 MB)

 Trainable params: 389,258 (1.48 MB)

 Non-trainable params: 1,088 (4.25 KB)

Epoch 1/50
782/782 - 30s - 39ms/step - accuracy: 0.3783 - loss: 1.7083 - val_accuracy: 0.5252 - val_loss: 1.3459 - learning_rate: 8.0000e-04
Epoch 2/50
782/782 - 7s - 9ms/step - accuracy: 0.5444 - loss: 1.2783 - val_accuracy: 0.6457 - val_loss: 0.9978 - learning_rate: 8.0000e-04
Epoch 3/50
782/782 - 7s - 9ms/step - accuracy: 0.6297 - loss: 1.0642 - val_accuracy: 0.6804 - val_loss: 0.8983 - learning_rate: 8.0000e-04
Epoch 4/50
782/782 - 7s - 9ms/step - accuracy: 0.6781 - loss: 0.9310 - val_accuracy: 0.7435 - val_loss: 0.7385 - learning_rate: 8.0000e-04
Epoch 5/50
782/782 - 7s - 9ms/step - accuracy: 0.7134 - loss: 0.8388 - val_accuracy: 0.7215 - val_loss: 0.8163 - learning_rate: 8.0000e-04
Epoch 6/50
782/782 - 7s - 9ms/step - accuracy: 0.7383 - loss: 0.7712 - val_accuracy: 0.7747 - val_loss: 0.6601 - learning_rate: 8.0000e-04
Epoch 7/50
782/782 - 7s - 9ms/step - accuracy: 0.7585 - loss: 0.7126 - val_accuracy: 0.7897 - val_loss: 0.6070 - learning_rate: 8.0000e-04
Epoch 8/50
782/782 - 7s -

Test loss: 0.5058  Test accuracy: 0.8652
